In [5]:
!pip install selenium

Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple


In [17]:
import pandas as pd 
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

driver_path = "/opt/homebrew/bin/chromedriver"

#1. Setting up chromedriver
service = Service(driver_path)
options = Options()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 Chrome/120 Safari/537.36")

driver = webdriver.Chrome(service=service, options=options)

In [23]:
#2. Getting the LinkedIn URL 
title = "Data%20Scientist"
location = "103291313"
start = 25
list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={title}&geoId={location}&start={start}"

# list_url = "https://www.linkedin.com/jobs/search/?currentJobId=4226976735&geoId=103291313&keywords=data%20scientist&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true"

driver.get(list_url)
time.sleep(3)

In [26]:
#3. Collecting data
jobs_list = []
page = 1
max_pages = 3

while page <= max_pages:
    print(f"📄 Collecting data from page {page} ...")

    review_elements = driver.find_elements(By.CSS_SELECTOR, "li[data-hook='review']")
    for r in review_elements:
        try:
            # Company Name
            try:
                company_name = r.find_element(By.CSS_SELECTOR, "a[data-tracking-control-name='public_jobs_jserp-result_job-search-card-subtitle']").text
            except:
                company_name = r.find_element(By.CSS_SELECTOR, "a[data-tracking-control-name='public_jobs_jserp-result_job-search-card-subtitle']").text
            rating = float(re.search(r"(\d+(\.\d+)?)", company_name).group(1)) if company_name else None

            # Job Title
            title = r.find_element(By.CSS_SELECTOR, "[class='base-search-card__title']").text.strip()

            # Posted Date
            date = r.find_element(By.CSS_SELECTOR, "time[class='job-search-card__listdate']").text.strip()

            # Job Description
            # body = r.find_element(By.CSS_SELECTOR, "[class='mt4']").text.strip()

            jobs_list.append({
                "company": company_name,
                "title": title,
                "date": date,
                # "body": body
            })

        except Exception as e:
            print("❌ Failed with Parsing:", e)
            continue

    # Click next page
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "li.a-last a")
        next_button.click()
        page += 1
        time.sleep(3)
    except NoSuchElementException:
        print("⛔️ There's no more pages.")
        break

📄 Collecting data from page 1 ...
⛔️ There's no more pages.


In [27]:
# 5. 결과 보기
df = pd.DataFrame(jobs_list)
print(df)

Empty DataFrame
Columns: []
Index: []
